In [ ]:
import csv
import pandas as pd
import itertools
import math
import urllib3
from bs4 import BeautifulSoup
import json

In [ ]:
# Import aggregated data in json form and assign to dataframe

link='https://raw.githubusercontent.com/2020PB/police-brutality/data_build/all-locations-v2.json'
df_from_json = pd.read_json(link)

# Check the shape of the dataframe
df_from_json.shape

(1171, 4)

In [ ]:
# Look at dataframe's head
df_from_json.head()

,edit_at,help,updated_at,data
0,https://github.com/2020PB/police-brutality,ask @ubershmekel on twitter,2020-10-06 19:59:38.372930+00:00,{'links': [{'url': 'https://www.youtube.com/wa...
1,https://github.com/2020PB/police-brutality,ask @ubershmekel on twitter,2020-10-06 19:59:38.372930+00:00,{'links': [{'url': 'https://mobile.twitter.com...
2,https://github.com/2020PB/police-brutality,ask @ubershmekel on twitter,2020-10-06 19:59:38.372930+00:00,{'links': [{'url': 'https://twitter.com/gunduz...
3,https://github.com/2020PB/police-brutality,ask @ubershmekel on twitter,2020-10-06 19:59:38.372930+00:00,{'links': [{'url': 'https://www.reddit.com/r/B...
4,https://github.com/2020PB/police-brutality,ask @ubershmekel on twitter,2020-10-06 19:59:38.372930+00:00,{'links': [{'url': 'https://www.fox10phoenix.c...


In [ ]:
# Look at dataframe's tail
df_from_json.tail()

,edit_at,help,updated_at,data
1166,https://github.com/2020PB/police-brutality,ask @ubershmekel on twitter,2020-10-06 19:59:38.372930+00:00,{'links': [{'url': 'https://www.pinknews.co.uk...
1167,https://github.com/2020PB/police-brutality,ask @ubershmekel on twitter,2020-10-06 19:59:38.372930+00:00,{'links': [{'url': 'https://twitter.com/greg_d...
1168,https://github.com/2020PB/police-brutality,ask @ubershmekel on twitter,2020-10-06 19:59:38.372930+00:00,{'links': [{'url': 'https://www.instagram.com/...
1169,https://github.com/2020PB/police-brutality,ask @ubershmekel on twitter,2020-10-06 19:59:38.372930+00:00,{'links': [{'url': 'https://twitter.com/shelby...
1170,https://github.com/2020PB/police-brutality,ask @ubershmekel on twitter,2020-10-06 19:59:38.372930+00:00,{'links': [{'url': 'https://twitter.com/DJPAIM...


In [ ]:
# Pull data column out into its own dataframe
df_2020PB = pd.json_normalize(df_from_json['data'])

# Ensure that dataframe was created correctly
df_2020PB.head()

,links,state,edit_at,city,description,tags,geolocation,name,date,date_text,id
0,[{'url': 'https://www.youtube.com/watch?v=s7MM...,Washington,https://github.com/2020PB/police-brutality/blo...,Olympia,Footage shows a few individuals break off from...,"[arrest, less-lethal, projectile, protester, s...",,Police respond to broken windows with excessiv...,2020-05-31,May 31st,wa-olympia-1
1,[{'url': 'https://mobile.twitter.com/chadloder...,Washington,https://github.com/2020PB/police-brutality/blo...,Seattle,Officer pins protester with his knee on his ne...,"[arrest, knee-on-neck, protester]",,Officer pins protester by pushing his knee int...,2020-05-30,May 30th,wa-seattle-1
2,[{'url': 'https://twitter.com/gunduzbaba1905/s...,Washington,https://github.com/2020PB/police-brutality/blo...,Seattle,A couple of police officers are seen beating a...,"[beat, protester, punch]",,Police beat unarmed man on the ground,2020-05-31,May 31st,wa-seattle-2
3,[{'url': 'https://www.reddit.com/r/Bad_Cop_No_...,Washington,https://github.com/2020PB/police-brutality/blo...,Seattle,A police officer randomly and indiscriminately...,"[less-lethal, pepper-spray, protester]",,Police indiscriminately pepper spray peaceful ...,2020-05-31,May 31st,wa-seattle-3
4,[{'url': 'https://www.fox10phoenix.com/news/vi...,Washington,https://github.com/2020PB/police-brutality/blo...,Seattle,Police pepper sprays a young child who is seen...,"[child, inhumane-treatment, less-lethal, peppe...",,Police pepper spray young child,2020-05-31,May 31st,wa-seattle-4


In [ ]:
# Clean dataframe

# Drop extraneous 'edit_at' column
df_2020PB = df_2020PB.drop(columns=['edit_at'])

# Write function to create hyperlinks for the 'links' columns
def cleanlinks(json):
    links_out = []
    for link in json:
        links_out.append(link['url'])
    return links_out

# Apply function to the dataframe 'links' column
df_2020PB['links'] = df_2020PB['links'].apply(cleanlinks)

# Update the "date" column to timestamps
df_2020PB['date'] = pd.to_datetime(df_2020PB['date'],format='%Y-%m-%d')

# Check the dataframe's top 5 rows
df_2020PB.head()

,links,state,city,description,tags,geolocation,name,date,date_text,id
0,[https://www.youtube.com/watch?v=s7MM1VauRHo],Washington,Olympia,Footage shows a few individuals break off from...,"[arrest, less-lethal, projectile, protester, s...",,Police respond to broken windows with excessiv...,2020-05-31,May 31st,wa-olympia-1
1,[https://mobile.twitter.com/chadloder/status/1...,Washington,Seattle,Officer pins protester with his knee on his ne...,"[arrest, knee-on-neck, protester]",,Officer pins protester by pushing his knee int...,2020-05-30,May 30th,wa-seattle-1
2,[https://twitter.com/gunduzbaba1905/status/126...,Washington,Seattle,A couple of police officers are seen beating a...,"[beat, protester, punch]",,Police beat unarmed man on the ground,2020-05-31,May 31st,wa-seattle-2
3,[https://www.reddit.com/r/Bad_Cop_No_Donut/com...,Washington,Seattle,A police officer randomly and indiscriminately...,"[less-lethal, pepper-spray, protester]",,Police indiscriminately pepper spray peaceful ...,2020-05-31,May 31st,wa-seattle-3
4,[https://www.fox10phoenix.com/news/video-shows...,Washington,Seattle,Police pepper sprays a young child who is seen...,"[child, inhumane-treatment, less-lethal, peppe...",,Police pepper spray young child,2020-05-31,May 31st,wa-seattle-4


In [ ]:
# Extract and clean the data from the 846 API

url="https://api.846policebrutality.com/api/incidents"
http = urllib3.PoolManager()
response = http.request('GET', url)
soup = BeautifulSoup(response.data, "html.parser")

json_846 = json.loads(soup.text)

json_846

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'data': [{'city': 'Portland',
   'data': None,
   'date': '2020-10-04T07:00:00.000000Z',
   'description': None,
   'geocoding': {'lat': '45.5051064', 'long': '-122.6750261'},
   'id': '9bab3230-07fc-11eb-91ba-8b0dc6ee5fae',
   'links': ['https://twitter.com/PDocumentarians/status/1312997624631357445'],
   'pb_id': 'or-portland-385',
   'state': 'Oregon',
   'tags': ['less-lethal', 'projectile', 'protester', 'shoot'],
   'title': 'Protester shot by sheriff deputy'},
  {'city': 'Portland',
   'data': None,
   'date': '2020-10-04T07:00:00.000000Z',
   'description': None,
   'geocoding': {'lat': '45.5051064', 'long': '-122.6750261'},
   'id': '9c0614c0-07fc-11eb-a9f0-e1824a38a51c',
   'links': ['https://twitter.com/MaranieRae/status/1313009265267732480',
    'https://twitter.com/MrOlmos/status/1313004031606833152',
    'https://twitter.com/suzettesmith/status/1313006462042480645',
    'https://twitter.com/1misanthrophile/status/1313006706687799296',
    'https://twitter.com/PdxCoparazzi

In [ ]:
# Check length of the json_846 file

len(json_846['data'])

1197

In [ ]:
# Retrieve data from the json_846 'data' key

incidents = json_846['data']

# Check out the first incidents entry

incidents[0]

{'city': 'Portland',
 'data': None,
 'date': '2020-10-04T07:00:00.000000Z',
 'description': None,
 'geocoding': {'lat': '45.5051064', 'long': '-122.6750261'},
 'id': '9bab3230-07fc-11eb-91ba-8b0dc6ee5fae',
 'links': ['https://twitter.com/PDocumentarians/status/1312997624631357445'],
 'pb_id': 'or-portland-385',
 'state': 'Oregon',
 'tags': ['less-lethal', 'projectile', 'protester', 'shoot'],
 'title': 'Protester shot by sheriff deputy'}

In [ ]:
# Create dataframe from the 846 API incident data
df_846 = pd.DataFrame(incidents)

# Change data type for 'date' column to datetime type
df_846['date'] = pd.to_datetime(df_846['date'], infer_datetime_format=True)

# Drop empty columns
df_846 = df_846.drop(columns=['data','description'])

# Check the top 5 rows of the new dataframe
df_846.head()

(1197, 11)
datetime64[ns]


,id,pb_id,state,city,date,title,links,tags,geocoding
0,9bab3230-07fc-11eb-91ba-8b0dc6ee5fae,or-portland-385,Oregon,Portland,2020-10-04 07:00:00,Protester shot by sheriff deputy,[https://twitter.com/PDocumentarians/status/13...,"[less-lethal, projectile, protester, shoot]","{'lat': '45.5051064', 'long': '-122.6750261'}"
1,9c0614c0-07fc-11eb-a9f0-e1824a38a51c,or-portland-386,Oregon,Portland,2020-10-04 07:00:00,Police rip protesters off sidewalk for arrest,[https://twitter.com/MaranieRae/status/1313009...,"[arrest, protester, push, shove, throw]","{'lat': '45.5051064', 'long': '-122.6750261'}"
2,f8bd14f0-078d-11eb-aeb4-99fc55dcca94,dc-dc-38,Washington DC,DC,2020-10-03 07:00:00,Police wrestle with protesters,[https://twitter.com/rawsmedia/status/13124477...,"[arrest, protester, push, shove, tackle]","{'lat': '38.9071923', 'long': '-77.0368707'}"
3,0a8148d0-078e-11eb-b971-338df9f33731,wa-seattle-72,Washington,Seattle,2020-10-03 07:00:00,"Police shove, shoot at, and arrest protesters",[https://twitter.com/MNMissy/status/1312625065...,"[arrest, bike, journalist, less-lethal, pepper...","{'lat': '47.6062095', 'long': '-122.3320708'}"
4,0bbf1b70-078e-11eb-aaa9-2fe2283446df,wa-seattle-73,Washington,Seattle,2020-10-03 07:00:00,Police arrest bystander in minivan during protest,[https://twitter.com/iNFoPiG2/status/131286028...,"[arrest, bystander]","{'lat': '47.6062095', 'long': '-122.3320708'}"
